In [4]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

# Define the problem instance
num_items = 44
bin_capacity = 1000
mutation_rate = 0.001
crossover_rate = 0.6
generations = 10

item_weights = {
    200: 5, 199: 2, 198: 2, 197: 2, 196: 1, 195: 3, 194: 2, 193: 2, 192: 4, 191: 2, 
    190: 4, 188: 3, 187: 2, 186: 2, 185: 1, 184: 1, 183: 1, 182: 1, 181: 3, 180: 1, 
    178: 3, 177: 2, 176: 2, 174: 1, 173: 1, 172: 1, 171: 3, 168: 2, 167: 1, 165: 1, 
    164: 1, 163: 1, 162: 3, 161: 3, 160: 3, 159: 2, 158: 3, 157: 3, 156: 2, 155: 5, 
    154: 3, 153: 3, 151: 5, 150: 2
}

print(item_weights)


def binary_to_decimal(binary_list):
    decimal_list = []
    for binary_nums in binary_list:
        decimal_nums = []
        for binary_num in binary_nums:
            if isinstance(binary_num, list):
                binary_num = ''.join([str(bit) for bit in binary_num])
            decimal_num = int(str(binary_num), 2)
            decimal_nums.append(decimal_num)
        decimal_list.append(decimal_nums)
    return decimal_list

def decimal_to_binary(decimal_list):
    bits = ''
    bits_list = []
    for sublist in decimal_list:
        sublist_bits = []
        for num in sublist:
            sublist_bits.append(format(num, '08b'))
        bits_list.append(sublist_bits)

    binary_list = '[' + ', '.join(['[' + ', '.join(sublist) + ']' for sublist in bits_list]) + ']'
    return binary_list
    
def sum_subsets(subsets):
    sums = []
    for subset in subsets:
        subset_sum = sum(subset)
        if subset_sum > bin_capacity:
            subset_sum = 0
        sums.append(subset_sum)
    return sums


def mutate_individual(individual, mutation_rate):
    mutated_individual = individual.copy()
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            # Flip the bit at position i
            mutated_individual[i] = 1 - mutated_individual[i]
    return mutated_individual

def mutate_population(population, mutation_rate):
    mutated_population = []
    for individual in population:
        mutated_individual = mutate_individual(individual, mutation_rate)
        mutated_population.append(mutated_individual)
    return mutated_population

# Function for crossover
def crossover(parent1, parent2):
    if np.random.rand() < crossover_rate:
        crossover_point = np.random.randint(1, 7)
        child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        return child1, child2
    else:
        return parent1, parent2

{200: 5, 199: 2, 198: 2, 197: 2, 196: 1, 195: 3, 194: 2, 193: 2, 192: 4, 191: 2, 190: 4, 188: 3, 187: 2, 186: 2, 185: 1, 184: 1, 183: 1, 182: 1, 181: 3, 180: 1, 178: 3, 177: 2, 176: 2, 174: 1, 173: 1, 172: 1, 171: 3, 168: 2, 167: 1, 165: 1, 164: 1, 163: 1, 162: 3, 161: 3, 160: 3, 159: 2, 158: 3, 157: 3, 156: 2, 155: 5, 154: 3, 153: 3, 151: 5, 150: 2}


In [5]:
def pick_random_integer():
    choices = list(item_weights.keys())
    weights = list(item_weights.values())
    random_integer = random.choices(choices, weights=weights)[0]
    return random_integer

def pick_integers():
    chosen_integers = []
    sublists = [[] for _ in range(num_items)]  # Create 44 empty sublists
    for _ in range(len(item_weights)):
        random_integer = pick_random_integer()
        chosen_integers.append(random_integer)
        sublist_index = len(chosen_integers) % num_items  # Determine the index of the sublist
        sublists[sublist_index].append(random_integer)  # Append the integer to the corresponding sublist
        item_weights[random_integer] -= 1  # Decrease the count of the chosen integer
        if item_weights[random_integer] == 0:
            del item_weights[random_integer]  # Remove the integer if its count becomes 0
            
    # Append remaining integers to sublists in sequence
    remaining_integers = list(item_weights.keys())
    for i, integer in enumerate(remaining_integers):
        sublist_index = i % 44
        sublists[sublist_index].append(integer)
        
    return sublists

In [6]:
chosen_sublists = pick_integers()
print("Chosen sublists:", chosen_sublists)

# Fuction to calculate fitness
def calculate_fitness(population):
        indiv_fitness = sum_subsets(population)
        return indiv_fitness 

Chosen sublists: [[151, 200], [194, 198], [158, 197], [198, 196], [158, 195], [191, 194], [154, 193], [160, 192], [190, 191], [171, 190], [195, 188], [192, 187], [190, 186], [199, 183], [155, 181], [157, 180], [156, 178], [177, 176], [177, 174], [161, 173], [171, 171], [181, 168], [186, 167], [187, 165], [184, 164], [155, 163], [150, 162], [178, 161], [193, 159], [160, 158], [160, 157], [182, 156], [190, 155], [162, 154], [200, 153], [200, 151], [185, 150], [154], [197], [159], [192], [172], [199], [153]]


In [7]:
# Main genetic algorithm loop
for generation in range(generations):
    fitness_values = calculate_fitness(chosen_sublists)
    fitness_bin_history = len(chosen_sublists)
    print("Bins used: " + str(fitness_bin_history))
    #fitness_bin_history.append(fitness_values)

   # Convert fitness values to probabilities
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]

    # Select parents based on fitness
    parents_indices = np.random.choice(len(chosen_sublists), size=len(chosen_sublists) // 2, p=probabilities)
    parents = [chosen_sublists[i] for i in parents_indices]

    # Create next generation through crossover and mutation
    children = []
    for i in range(0, 5):
        child1, child2 = crossover(parents[i], parents[i + 1])
        child1 = mutate_individual(child1, mutation_rate)
        child2 = mutate_individual(child2, mutation_rate)
        children.extend([child1, child2])

    chosen_sublists[:len(chosen_sublists) // 2] = parents
    chosen_sublists[len(chosen_sublists) // 2:] = children

Bins used: 44
Bins used: 32
Bins used: 26
Bins used: 23
Bins used: 21
Bins used: 20
Bins used: 20
Bins used: 20
Bins used: 20
Bins used: 20
